<a href="https://colab.research.google.com/github/LakshmiMedapati96/Duplicate/blob/main/DM_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading the dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! mkdir ~/.kaggle

In [ ]:
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download tushar5harma/topredditcomments

 41% 9.00M/22.1M [00:00<00:00, 88.9MB/s]
100% 22.1M/22.1M [00:00<00:00, 128MB/s] 


In [ ]:
! unzip topredditcomments.zip

Archive:  topredditcomments.zip
  inflating: Top_Posts.csv           
  inflating: Top_Posts_Comments.csv  


# The Coding begins...

# Importing Libraries, Loading and Merging the data files

**Importing Libraries**

In [ ]:
import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import string
import re

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import GridSearchCV

from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

**Loading the dataset**

In [ ]:
# Load the dataset
posts = pd.read_csv("Top_Posts.csv")
comments = pd.read_csv("Top_Posts_Comments.csv")

print("posts", posts.shape)
print("comments", comments.shape)

posts (2987, 10)
comments (223174, 2)


In [ ]:
posts.head()

,post_id,post_title,subreddit,post_url,flair_text,score,comments,upvote_ratio,date-time,year
0,gh1dj9,[Project] From books to presentations in 10s w...,MachineLearning,https://v.redd.it/v492uoheuxx41,Project,7798,186,0.99,2020-05-10 13:19:54,2020
1,kuc6tz,[D] A Demo from 1993 of 32-year-old Yann LeCun...,MachineLearning,https://v.redd.it/25nxi9ojfha61,Discussion,5851,133,0.98,2021-01-10 10:30:36,2021
2,g7nfvb,[R] First Order Motion Model applied to animat...,MachineLearning,https://v.redd.it/rlmmjm1q5wu41,Research,4761,111,0.97,2020-04-25 04:27:23,2020
3,lui92h,[N] AI can turn old photos into moving Images ...,MachineLearning,https://v.redd.it/ikd5gjlbi8k61,News,4688,230,0.97,2021-02-28 15:12:28,2021
4,ohxnts,[D] This AI reveals how much time politicians ...,MachineLearning,https://i.redd.it/34sgziebfia71.jpg,Discussion,4568,228,0.96,2021-07-11 04:18:59,2021


In [ ]:
comments.head()

,post_id,comment
0,gh1dj9,Twitter thread: [https://twitter.com/cyrildiag...
1,gh1dj9,The future 🤯
2,gh1dj9,Simple yet very useful. Thank you for sharing ...
3,gh1dj9,"Almost guaranteed, Apple will copy your idea i..."
4,gh1dj9,Ohh the nightmare of making this into a stable...


In [ ]:
posts.subreddit.value_counts()

artificial         999
MachineLearning    998
datascience        990
Name: subreddit, dtype: int64

**Merging the data frames into a single pandas dataframe**

In [ ]:
# The code in this cell was written by me by referring to Reference [1].

# Merging the 2 input files by using the common column "post_id" in both the files
merged_data = pd.merge(posts, comments, on="post_id", how="inner")

merged_data

,post_id,post_title,subreddit,post_url,flair_text,score,comments,upvote_ratio,date-time,year,comment
0,gh1dj9,[Project] From books to presentations in 10s w...,MachineLearning,https://v.redd.it/v492uoheuxx41,Project,7798,186,0.99,2020-05-10 13:19:54,2020,Twitter thread: [https://twitter.com/cyrildiag...
1,gh1dj9,[Project] From books to presentations in 10s w...,MachineLearning,https://v.redd.it/v492uoheuxx41,Project,7798,186,0.99,2020-05-10 13:19:54,2020,The future 🤯
2,gh1dj9,[Project] From books to presentations in 10s w...,MachineLearning,https://v.redd.it/v492uoheuxx41,Project,7798,186,0.99,2020-05-10 13:19:54,2020,Simple yet very useful. Thank you for sharing ...
3,gh1dj9,[Project] From books to presentations in 10s w...,MachineLearning,https://v.redd.it/v492uoheuxx41,Project,7798,186,0.99,2020-05-10 13:19:54,2020,"Almost guaranteed, Apple will copy your idea i..."
4,gh1dj9,[Project] From books to presentations in 10s w...,MachineLearning,https://v.redd.it/v492uoheuxx41,Project,7798,186,0.99,2020-05-10 13:19:54,2020,Ohh the nightmare of making this into a stable...
...,...,...,...,...,...,...,...,...,...,...,...
223163,efk5n3,Tesla's Neural Net can now identify red and gr...,artificial,https://www.teslarati.com/tesla-holiday-update...,NaN,80,10,0.89,2019-12-25 18:50:50,2019,LiDAR is mot powerful sensor for the auto driv...
223164,efk5n3,Tesla's Neural Net can now identify red and gr...,artificial,https://www.teslarati.com/tesla-holiday-update...,NaN,80,10,0.89,2019-12-25 18:50:50,2019,So it can now idenrify traffic lights? Musk pr...
223165,efk5n3,Tesla's Neural Net can now identify red and gr...,artificial,https://www.teslarati.com/tesla-holiday-update...,NaN,80,10,0.89,2019-12-25 18:50:50,2019,Hydranet bro!
223166,efk5n3,Tesla's Neural Net can now identify red and gr...,artificial,https://www.teslarati.com/tesla-holiday-update...,NaN,80,10,0.89,2019-12-25 18:50:50,2019,It even shows flashing yellow turn arrows.


In [ ]:
# Storing the necessary columns only
merged_data = merged_data[["comment", "subreddit"]]

merged_data.head()

,comment,subreddit
0,Twitter thread: [https://twitter.com/cyrildiag...,MachineLearning
1,The future 🤯,MachineLearning
2,Simple yet very useful. Thank you for sharing ...,MachineLearning
3,"Almost guaranteed, Apple will copy your idea i...",MachineLearning
4,Ohh the nightmare of making this into a stable...,MachineLearning


# Data Preprocessing

In [ ]:
print(merged_data.shape)

# checking for missing values
merged_data.isnull().sum()


(223168, 2)


comment      9
subreddit    0
dtype: int64

In [ ]:
# Dropping the missing values
merged_data.dropna(inplace=True)

print(merged_data.shape)

(223159, 2)


<ipython-input-16-2b16886c28c2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data.dropna(inplace=True)


In [ ]:
# The below code is taken from Reference [2].

# Defining a function to remove URLs
def remove_urls(doc):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', doc)

In [ ]:
# The below code is taken from Reference [3].

# Defining a function to remove emoji
def remove_emoji(doc):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"          # emoticons
        u"\U0001F300-\U0001F5FF"          # symbols & pictographs
        u"\U0001F680-\U0001F6FF"          # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"          # flags (iOS)
        u"\U00002500-\U00002BEF"          # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"                         # dingbats
        u"\u3030"
        "]+", re.UNICODE)
    return emoji_pattern.sub(r'', doc)

In [ ]:
 stop_words = stopwords.words('english')           # Accessing stopwords from nltk library

lemmatizer = WordNetLemmatizer()                  # Lemmatization

# Defining a function to clean the data
def clean(doc):
    doc = str(doc)                                              # converting to string 
    doc = doc.lower().strip()                                   # converting to lower case and stripping the whitespaces
    doc = "".join([char for char in doc if char not in string.punctuation and not char.isdigit()])          # removing punctuations and digits

     doc = " ".join([token for token in doc.split() if token not in stop_words])                 # splitting the doc into tokens and removing stopwords
  
    doc = "".join([lemmatizer.lemmatize(word) for word in doc])                                 # lemmatizing the doc
    return doc

In [ ]:
# Cleaning the dataset
merged_data["comment"] = merged_data["comment"].apply(remove_urls)  
merged_data["comment"] = merged_data["comment"].apply(remove_emoji)          
merged_data["comment"] = merged_data["comment"].apply(clean)          

<ipython-input-20-af3d8f411adc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data["comment"] = merged_data["comment"].apply(remove_urls)
<ipython-input-20-af3d8f411adc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data["comment"] = merged_data["comment"].apply(remove_emoji)
<ipython-input-20-af3d8f411adc>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

In [ ]:
merged_data.head()

,comment,subreddit
0,twitter thread code background removal done un...,MachineLearning
1,future,MachineLearning
2,simple yet useful thank sharing code,MachineLearning
3,almost guaranteed apple copy idea,MachineLearning
4,ohh nightmare making stable product enough dri...,MachineLearning


**Dividing the dataset into Train, Validation and Test sets - 60:20:20 ratio**

In [ ]:
# print(merged_data.head())
# print(type(merged_data))
X = merged_data['comment']
y = merged_data['subreddit']

# Split the data into training and remaining datasets
X_trainset, X_remset, y_trainset, y_remset = train_test_split(X, y, train_size = 0.6,  random_state=42)

# Split the remaining dataset into validation and test datasets
X_valset, X_testset, y_valset, y_testset = train_test_split(X_remset, y_remset, train_size = 0.5,  random_state=42)


print(X_trainset.shape, y_trainset.shape)
print(X_valset.shape, y_valset.shape)
print(X_testset.shape, y_testset.shape)

(133895,) (133895,)
(44632,) (44632,)
(44632,) (44632,)


In [ ]:
# Written by me by referring to Reference [6].

# Vectorize dataset 
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_trainset)
X_val = vectorizer.transform(X_valset)
X_test = vectorizer.transform(X_testset)
y_train = vectorizer.fit_transform(y_trainset)
y_val = vectorizer.transform(y_valset)
y_test = vectorizer.transform(y_testset)

# **Model Building**

# **Model 1 :** Decision Tree

In [ ]:
# Written by me by referring to Reference [6].

dtc = DecisionTreeClassifier(max_depth = 3)
dtc = dtc.fit(X_train, y_trainset)

y_pred_train = dtc.predict(X_train)
y_pred_val = dtc.predict(X_val) 
y_pred_test = dtc.predict(X_test)

accuracy_val_DT_BHT = accuracy_score(y_valset, y_pred_val)
accuracy_test_DT_BHT = accuracy_score(y_testset, y_pred_test)
print("Accuracy on validation set before hyperparameter tuning :",accuracy_val_DT_BHT)
print("Accuracy on test set before hyperparameter tuning:",accuracy_test_DT_BHT)

Accuracy on validation set before hyperparameter tuning : 0.5261471589890662
Accuracy on test set before hyperparameter tuning: 0.5265952679691701


**Hyperparameter Tuning with GridSearchCV**

In [ ]:
# Written by me by referring to Reference [7] and [8].

param_dict = {
    "max_depth": range(13, 15),
    "min_samples_split": [4, 5, 6]
}

grid_search_dt = GridSearchCV(dtc, param_dict, cv = 10)

grid_search_dt.fit(X_train, y_trainset)
print("Best hyperparameters:", grid_search_dt.best_params_)


accuracy_val_DT_AHT = grid_search_dt.best_estimator_.score(X_val, y_valset)
accuracy_test_DT_AHT = grid_search_dt.best_estimator_.score(X_test, y_testset)
print("Accuracy on validation set after hyperparameter tuning : ", accuracy_val_DT_AHT)
print("Final accuracy on test set after hyperparameter tuning : ", accuracy_test_DT_AHT)

Best hyperparameters: {'max_depth': 14, 'min_samples_split': 4}
Accuracy on validation set after hyperparameter tuning :  0.5699498117942283
Final accuracy on test set after hyperparameter tuning :  0.5669250761785266


**Classification Report**

In [ ]:
# Written by me by referring to Reference [11].

prediction = grid_search_dt.best_estimator_
prediction = prediction.predict(X_test)
print(classification_report(y_testset, prediction, target_names=['MachineLearning', 'datascience','artificial']))

                 precision    recall  f1-score   support

MachineLearning       0.50      0.93      0.65     19139
    datascience       0.26      0.00      0.01      3985
     artificial       0.84      0.35      0.49     21508

       accuracy                           0.57     44632
      macro avg       0.53      0.43      0.38     44632
   weighted avg       0.64      0.57      0.52     44632



# **Model 2 :** Random Forest

In [ ]:
# Written by me by referring to Reference [9].

rfc = RandomForestClassifier(n_estimators = 110, max_depth = 2)
rfc = rfc.fit(X_train, y_trainset)

y_pred_train = rfc.predict(X_train)
y_pred_val = rfc.predict(X_val) 
y_pred_test = rfc.predict(X_test)

accuracy_val_RF_BHT = accuracy_score(y_valset, y_pred_val)
accuracy_test_RF_BHT = accuracy_score(y_testset, y_pred_test)
print("Accuracy on validation set before hyperparameter tuning :",accuracy_val_RF_BHT)
print("Accuracy on test set before hyperparameter tuning:",accuracy_test_RF_BHT)

Accuracy on validation set before hyperparameter tuning : 0.4818739917547948
Accuracy on test set before hyperparameter tuning: 0.48243412797992474


**Hyperparameter Tuning with GridSearchCV**

In [ ]:
# Written by me by referring to Reference [7] and [8].

param_dict = {
    "n_estimators" : [25, 50, 100],
    "max_depth": range(13, 15)  
}

grid_search_rf = GridSearchCV(rfc, param_dict, cv = 10)

grid_search_rf.fit(X_train, y_trainset)
print("Best hyperparameters:", grid_search_rf.best_params_)


accuracy_val_RF_AHT = grid_search_rf.best_estimator_.score(X_val, y_valset)
accuracy_test_RF_AHT = grid_search_rf.best_estimator_.score(X_test, y_testset)
print("Accuracy on validation set after hyperparameter tuning : ", accuracy_val_RF_AHT)
print("Final accuracy on test set after hyperparameter tuning : ", accuracy_test_RF_AHT)

Best hyperparameters: {'max_depth': 14, 'n_estimators': 100}
Accuracy on validation set after hyperparameter tuning :  0.5764025811077254
Final accuracy on test set after hyperparameter tuning :  0.578844775049292


**Classification Report**

In [ ]:
# Written by me by referring to Reference [11].

prediction = grid_search_rf.best_estimator_
prediction = prediction.predict(X_test)
print(classification_report(y_testset, prediction, target_names=['MachineLearning', 'datascience','artificial']))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                 precision    recall  f1-score   support

MachineLearning       0.81      0.24      0.37     19139
    datascience       0.00      0.00      0.00      3985
     artificial       0.54      0.99      0.70     21508

       accuracy                           0.58     44632
      macro avg       0.45      0.41      0.36     44632
   weighted avg       0.61      0.58      0.50     44632



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# **Model 3 :** K - Nearest Neighbors

In [ ]:
# Written by me by referring to Reference [10].

knc = KNeighborsClassifier(n_neighbors = 3)
knc = knc.fit(X_train, y_trainset)

y_pred_train = knc.predict(X_train)
y_pred_val = knc.predict(X_val) 
y_pred_test = knc.predict(X_test)

accuracy_val_KNN_BHT = accuracy_score(y_valset, y_pred_val)
accuracy_test_KNN_BHT = accuracy_score(y_testset, y_pred_test)
print("Accuracy on validation set before hyperparameter tuning :",accuracy_val_KNN_BHT)
print("Accuracy on test set before hyperparameter tuning:",accuracy_test_KNN_BHT)

Accuracy on validation set before hyperparameter tuning : 0.5440267072952142
Accuracy on test set before hyperparameter tuning: 0.5474995518910198


**Hyperparameter Tuning with GridSearchCV**

In [ ]:
# Written by me by referring to Reference [7] and [8].

param_dict = {
    "n_neighbors" : [5, 7, 11]  
}

grid_search_kn = GridSearchCV(knc, param_dict, cv = 10)

grid_search_kn.fit(X_train, y_trainset)
print("Best hyperparameters:", grid_search_kn.best_params_)


accuracy_val_KNN_AHT = grid_search_kn.best_estimator_.score(X_val, y_valset)
accuracy_test_KNN_AHT = grid_search_kn.best_estimator_.score(X_test, y_testset)
print("Accuracy on validation set after hyperparameter tuning : ", accuracy_val_KNN_AHT)
print("Final accuracy on test set after hyperparameter tuning : ", accuracy_test_KNN_AHT)

Best hyperparameters: {'n_neighbors': 11}
Accuracy on validation set after hyperparameter tuning :  0.5660288582183187
Final accuracy on test set after hyperparameter tuning :  0.5628472844595805


**Classification Report**

In [ ]:
# Written by me by referring to Reference [11].

prediction = grid_search_kn.best_estimator_
prediction = prediction.predict(X_test)
print(classification_report(y_testset, prediction, target_names=['MachineLearning', 'datascience','artificial']))

                 precision    recall  f1-score   support

MachineLearning       0.54      0.54      0.54     19139
    datascience       0.26      0.07      0.11      3985
     artificial       0.59      0.67      0.63     21508

       accuracy                           0.56     44632
      macro avg       0.47      0.43      0.43     44632
   weighted avg       0.54      0.56      0.55     44632



# **Model 4 :** Logistic Regression

In [ ]:
# Written by me by referring to Reference [12].

lg = LogisticRegression(C = 5.6, max_iter = 2000)
lg = lg.fit(X_train, y_trainset)

y_pred_train = lg.predict(X_train)
y_pred_val = lg.predict(X_val) 
y_pred_test = lg.predict(X_test)

accuracy_val_LG_BHT = accuracy_score(y_valset, y_pred_val)
accuracy_test_LG_BHT = accuracy_score(y_testset, y_pred_test)
print("Accuracy on validation set before hyperparameter tuning :",accuracy_val_LG_BHT)
print("Accuracy on test set before hyperparameter tuning:",accuracy_test_LG_BHT)

Accuracy on validation set before hyperparameter tuning : 0.6880489335006273
Accuracy on test set before hyperparameter tuning: 0.6836574654956086


**Hyperparameter Tuning with GridSearchCV**

In [ ]:
# Written by me by referring to Reference [7] and [8].

param_dict = {
    "C" : [0.01, 0.4, 1.3]  
}

grid_search_lg = GridSearchCV(lg, param_dict, cv = 10)

grid_search_lg.fit(X_train, y_trainset)
print("Best hyperparameters:", grid_search_lg.best_params_)


accuracy_val_LG_AHT = grid_search_lg.best_estimator_.score(X_val, y_valset)
accuracy_test_LG_AHT = grid_search_lg.best_estimator_.score(X_test, y_testset)
print("Accuracy on validation set after hyperparameter tuning : ", accuracy_val_LG_AHT)
print("Final accuracy on test set after hyperparameter tuning : ", accuracy_test_LG_AHT)

Best hyperparameters: {'C': 0.4}
Accuracy on validation set after hyperparameter tuning :  0.704404911274422
Final accuracy on test set after hyperparameter tuning :  0.7010440939236422


**Classification Report**

In [ ]:
# Written by me by referring to Reference [11].

prediction = grid_search_lg.best_estimator_
prediction = prediction.predict(X_test)
print(classification_report(y_testset, prediction, target_names=['MachineLearning', 'datascience','artificial']))

                 precision    recall  f1-score   support

MachineLearning       0.68      0.72      0.70     19139
    datascience       0.44      0.14      0.21      3985
     artificial       0.74      0.79      0.76     21508

       accuracy                           0.70     44632
      macro avg       0.62      0.55      0.56     44632
   weighted avg       0.68      0.70      0.68     44632



# **Model 5 :** Naive Bayes

**Build vocabulary as dictionary**

In [ ]:
# All the lines of code in this cell were written by me.

# Defining a function to build vocabulary as dictionary and 
# omitting rare words (if the occurrence is less than five times)
def build_vocab_dict(dataset, flag = False):
    vocab_dict = {}

    for review in dataset["comment"]:
        
        review = review.split(" ")
        # print(review)
        for word in review:
            word = word.lower()
            if word in vocab_dict:
                vocab_dict[word] += 1
            else:
                vocab_dict[word] = 1
                
    if flag:
        return vocab_dict
    total_vocab_dict = {k: v for k, v in vocab_dict.items() if v >= 5}
    return total_vocab_dict

**Total words dictionary - Training data**

In [ ]:
# All the lines of code in this cell were written by me.

#total_word_dict = build_vocab_dict(data)                        # dictionary for entire dataset
# create a DataFrame for X_train
X_train_df = pd.DataFrame(X_trainset, columns=["comment"])
# create a DataFrame for y_train
y_train_df = pd.DataFrame(y_trainset, columns=["subreddit"])

# combine X_train_df and y_train_df into a single DataFrame
train_df = pd.concat([X_train_df, y_train_df], axis=1)

total_word_dict_train = build_vocab_dict(train_df)              # dictionary for training dataset
print(f"Total words in the dictionary - training data set : {total_word_dict_train}")

Total words in the dictionary - training data set : {'joke': 474, 'post': 3129, 'reads': 144, 'bit': 2993, 'like': 24668, 'you’re': 1728, 'big': 3596, 'query': 534, 'getting': 3782, 'data': 32269, 'warehouse': 110, 'get': 15710, 'though': 4630, 'probably': 5009, 'system': 2395, 'engineered': 48, 'way': 9383, 'etl': 279, 'automated': 473, 'behind': 922, 'scenes': 99, 'companies': 4380, 'want': 8525, 'lot': 8535, 'legacy': 131, 'really': 11255, 'struggling': 194, 'even': 11477, 'understanding': 2271, 'analytics': 1576, 'suck': 204, 'theyre': 2561, 'necessarily': 722, 'complaining': 205, 'asking': 1395, 'productive': 266, 'ways': 1274, 'fill': 308, 'time': 12692, 'work': 17281, 'use': 11627, 'perhaps': 1064, 'could': 9332, 'offer': 939, 'prompts': 122, 'test': 2014, 'theory': 1559, 'someone': 5677, 'explain': 1425, 'limitations': 291, 'running': 1143, 'model': 8158, 'gpus': 496, 'cpus': 88, 'slow': 485, 'finish': 483, 'thanks': 4174, 'yay': 36, 'haha': 670, 'good': 12346, 'finding': 833, 

**Calculating prior probabilities**

In [ ]:
# All the lines of code in this cell were written by me.

# Creating separate dataframes for MachineLearning, datascience and artificial classes

ml_df = train_df[train_df["subreddit"] == 'MachineLearning']
ds_df = train_df[train_df["subreddit"] == 'datascience']
ai_df = train_df[train_df["subreddit"] == 'artificial']

#print(ml_df)
#print(ds_df)
#print(ai_df)

# Probabilities for MachineLearning, datascience and artificial classes

prob_ML = ml_df.shape[0]  / y_trainset.shape[0]
prob_DS = ds_df.shape[0]  / y_trainset.shape[0]
prob_AI = ai_df.shape[0]  / y_trainset.shape[0]

print(f"P(Machine Learning) = {str(prob_ML)}")
print(f"P(DataScience) = {str(prob_DS)}")
print(f"P(Artificial) = {str(prob_AI)}")

P(Machine Learning) = 0.4293812315620449
P(DataScience) = 0.4792113223047911
P(Artificial) = 0.09140744613316405


**Creating separate word dictionaries for MachineLearning, datascience and artificial classes**

In [ ]:
# All the lines of code in this cell were written by me.

# Machine Learning dictionary
ml_word_dict_train = build_vocab_dict(ml_df)
print(f"Words dictionary for machine learning label: {ml_word_dict_train}")

# Data Science dictionary
ds_word_dict_train = build_vocab_dict(ds_df)
print(f"Words dictionary for data science label: {ds_word_dict_train}")

# Artificial dictionary
ai_word_dict_train = build_vocab_dict(ai_df)
print(f"Words dictionary for artifical label: {ai_word_dict_train}")

Words dictionary for machine learning label: {'probably': 2109, 'someone': 2032, 'explain': 521, 'limitations': 152, 'running': 535, 'model': 4206, 'gpus': 450, 'cpus': 72, 'slow': 227, 'finish': 285, 'thanks': 2163, 'freaking': 30, 'love': 982, 'sweet': 58, 'thank': 999, 'isnt': 1809, 'odd': 64, 'want': 3213, 'use': 5084, 'one': 6798, 'posters': 39, 'reply': 256, 'reflect': 64, 'simultaneously': 67, 'arguing': 147, 'chinese': 358, 'citizens': 47, 'complaint': 44, 'shouldn’t': 57, 'ccp': 39, 'first': 2476, 'glance': 27, 'beginners': 68, 'book': 743, 'unless': 542, 'solid': 158, 'understanding': 923, 'mathematics': 218, 'statistics': 282, 'mean': 1858, 'youre': 2528, 'regular': 151, 'programmer': 87, 'math': 814, 'beyond': 333, 'hs': 10, 'curriculum': 32, 'tough': 87, 'read': 1817, 'better': 3156, 'served': 39, 'finding': 294, 'conceptual': 34, 'books': 288, 'learning': 5139, 'side': 632, 'return': 163, 'seems': 2119, 'directed': 41, 'graduate': 139, 'students': 584, 'lmao': 120, 'u': 1

**Calculating probability of the occurrence**

In [ ]:
# All the lines of code in this cell were written by me.

total_word_dict_train_tuple = dict()

# Calculating probability of all the words
total_num_of_words_train = sum(list(total_word_dict_train.values()))

for key, value in total_word_dict_train.items():
    total_word_dict_train_tuple[key] = (value, value*100/total_num_of_words_train)

print(f"Probability of the occurrence: {str(total_word_dict_train_tuple)}")

Probability of the occurrence: {'joke': (474, 0.01565295613011371), 'post': (3129, 0.10332932432726963), 'reads': (144, 0.004755328444591507), 'bit': (2993, 0.09883818079626654), 'like': (24668, 0.814614181049884), 'you’re': (1728, 0.05706394133509809), 'big': (3596, 0.11875111865799348), 'query': (534, 0.01763434298202684), 'getting': (3782, 0.12489341789892418), 'data': (32269, 1.065622872073079), 'warehouse': (110, 0.003632542561840735), 'get': (15710, 0.518793124059254), 'though': (4630, 0.15289701873929637), 'probably': (5009, 0.16541277902054763), 'system': (2395, 0.07909035850553237), 'engineered': (48, 0.0015851094815305024), 'way': (9383, 0.30985588052501467), 'etl': (279, 0.009213448861396046), 'automated': (473, 0.015619933015915159), 'behind': (922, 0.03044731129106507), 'scenes': (99, 0.0032692883056566614), 'companies': (4380, 0.14464124018965835), 'want': (8525, 0.28152204854265694), 'lot': (8535, 0.2818522796846425), 'legacy': (131, 0.0043260279600103296), 'really': (11

**Calculating conditional probability based on the category**

In [ ]:
# All the lines of code in this cell were written by me.

# Calculating probability of each word in MachineLearning.
ml_total_num_of_words = sum(list(ml_word_dict_train.values()))
ml_word_prob_dict_train = dict()
ml_word_prob_dict_train_tuple = dict()
for key, value in ml_word_dict_train.items():
    ml_word_prob_dict_train_tuple[key] = value*100/ml_total_num_of_words
print(f"Conditional probability based on 'Machine Learning' label: {str(ml_word_prob_dict_train_tuple)}")


# Calculating probability of each word in datascience.
ds_total_num_of_words = sum(list(ds_word_dict_train.values()))
ds_word_prob_dict_train = dict()
ds_word_prob_dict_train_tuple = dict()
for key, value in ds_word_dict_train.items():
    ds_word_prob_dict_train_tuple[key] = value*100/ds_total_num_of_words
print(f"Conditional probability based on 'Data Science' label: {str(ds_word_prob_dict_train_tuple)}")


# Calculating probability of each word in artificial.
ai_total_num_of_words = sum(list(ai_word_dict_train.values()))
ai_word_prob_dict_train = dict()
ai_word_prob_dict_train_tuple = dict()
for key, value in ai_word_dict_train.items():
    ai_word_prob_dict_train_tuple[key] = value*100/ai_total_num_of_words
print(f"Conditional probability based on 'artificial' label: {str(ai_word_prob_dict_train_tuple)}")

Conditional probability based on 'Machine Learning' label: {'probably': 0.1588353580008887, 'someone': 0.1530362482018994, 'explain': 0.039238132536018495, 'limitations': 0.011447593369433419, 'running': 0.04029251613583473, 'model': 0.31676695863050636, 'gpus': 0.03389090142266473, 'cpus': 0.005422544227626357, 'slow': 0.017096076939877542, 'finish': 0.02146423756768766, 'thanks': 0.16290226617160847, 'freaking': 0.0022593934281776485, 'love': 0.0739574782156817, 'sweet': 0.004368160627810121, 'thank': 0.0752378011583157, 'isnt': 0.13624142371911221, 'odd': 0.00482003931344565, 'want': 0.24198103615782615, 'use': 0.38289187296183885, 'one': 0.5119785508250552, 'posters': 0.002937211456630943, 'reply': 0.0192801572537826, 'reflect': 0.00482003931344565, 'simultaneously': 0.005045978656263416, 'arguing': 0.011071027798070478, 'chinese': 0.026962094909586606, 'citizens': 0.0035397163708116492, 'complaint': 0.0033137770279938845, 'shouldn’t': 0.004292847513537533, 'ccp': 0.002937211456630

**Calculating accuracy on Validation dataset**


In [ ]:
# All the lines of code in this cell were written by me.
                      
# create a DataFrame for X_valset and X_testset
X_val_df = pd.DataFrame(X_valset, columns=["comment"])
X_test_df = pd.DataFrame(X_testset, columns=["comment"])

# create a DataFrame for y_valset and y_testset
y_val_df = pd.DataFrame(y_valset, columns=["subreddit"])
y_test_df = pd.DataFrame(y_testset, columns=["subreddit"])

# combine X_val_df and y_val_df into a single DataFrame
val_df = pd.concat([X_val_df, y_val_df], axis=1)

# combine X_test_df and y_test_df into a single DataFrame
test_df = pd.concat([X_test_df, y_test_df], axis=1)

In [ ]:
# All the lines of code in this cell were taken from Reference [13].

# Defining a function to calculate accuracy score
def accuracyscore(actual, predicted):
    correct = 0
    # print(actual.shape[0])
    for i in range(actual.shape[0]):
        if actual["subreddit"].iloc[i] == predicted[i]:
            correct = correct + 1
            
        # print(actual[i] , predicted[i])
    #print(f"correct = {correct}")
    #print(str(rotten_word_prob_dict_train)[:200])
    return correct * 100.0/ float(len(actual))

In [ ]:
# Defining a function to predict the class label
def predict(preddata):
    pred = []
    for review in preddata["comment"]:
        vocab_dict = {}
        review = review.split(" ")
        # print(review)
        for word in review:
            word = word.lower()
            if word in vocab_dict:
                vocab_dict[word] += 1
            else:
                vocab_dict[word] = 1
        vocab_dict.pop('', None)                                     # for removing '' in the dictionary
        likelihood_ml = 1.0
        likelihood_ds = 1.0
        likelihood_ai = 1.0
        
        for word in vocab_dict:
            if not word in ml_word_dict_train:
                ml_word_prob_dict_train_tuple[word] = 0.0

            if not word in ds_word_dict_train:
                ds_word_prob_dict_train_tuple[word] = 0.0
            
            if not word in ai_word_dict_train:
                ai_word_prob_dict_train_tuple[word] = 0.0

            likelihood_ml = likelihood_ml * ml_word_prob_dict_train_tuple[word]
            likelihood_ds = likelihood_ds * ds_word_prob_dict_train_tuple[word]
            likelihood_ai = likelihood_ai * ai_word_prob_dict_train_tuple[word]
        
        posterior_ml = prob_ML * likelihood_ml
        posterior_ds = prob_DS * likelihood_ds
        posterior_ai = prob_AI * likelihood_ai

        max_var = max((posterior_ml, 'MachineLearning'), (posterior_ds, 'datascience'), (posterior_ai, 'artificial'))[1]
        pred.append(max_var)
    return pred

In [ ]:
# All the lines of code in this cell were taken from Reference [13].

val_pred = predict(val_df)
test_pred = predict(test_df)

accuracy_val = accuracyscore(val_df, val_pred)
print(f"Accuracy on validation set : {accuracy_val}%")

accuracy_test = accuracyscore(test_df, test_pred)
print(f"Accuracy on test set : {accuracy_test}%")

Accuracy on validation set : 57.779171894604765%
Accuracy on test set : 57.68955009858398%


**Classification Report**

In [ ]:
# Written by me by referring to Reference [11].

prediction = predict(test_df)
print(classification_report(y_testset, prediction, target_names=['MachineLearning', 'datascience','artificial']))

                 precision    recall  f1-score   support

MachineLearning       0.62      0.38      0.47     19139
    datascience       0.31      0.08      0.13      3985
     artificial       0.57      0.84      0.68     21508

       accuracy                           0.58     44632
      macro avg       0.50      0.44      0.43     44632
   weighted avg       0.57      0.58      0.54     44632



# Predicting a Comment

In [ ]:
# All the lines of code in this cell were written by me.
# showing for logistic regression.

import numpy as np
np.random.seed(48) 
#sampled_comments = val_df.sample(5)
#print(val_df.shape[0])
sampled_comments = val_df.sample(5)
print(sampled_comments["comment"].values.tolist())
print(sampled_comments["subreddit"].values.tolist())

pred_sample_comments = sampled_comments["subreddit"].values.tolist()
# print(pred_sample_comments)

['changed default stringsasfactors false r', 'joke openai clown show rename closedai', 'yeah simple script don’t think need sphnx', 'phone screen good place find job really generally figure role entails phone interview varies widely job description states raise red flag lines salary job description analyst scientist roles pretty blurry right experience', 'see specialized pytorchfocused env please ignore question']
['datascience', 'MachineLearning', 'datascience', 'datascience', 'MachineLearning']


In [ ]:
# All the lines of code in this cell were written by me.

from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

# define some example comments
comments = [
    'changed default stringsasfactors false r', 
    'joke openai clown show rename closedai', 
    'yeah simple script don’t think need sphnx', 
    'phone screen good place find job really generally figure role entails phone interview varies widely job description states raise red flag lines salary job description analyst scientist roles pretty blurry right experience', 
    'see specialized pytorchfocused env please ignore question']

# initialize and fit the CountVectorizer object on the training data
vectorizer = CountVectorizer()
vectorizer.fit(X_trainset)

# transform the test data using the fitted vectorizer
X_new = vectorizer.transform(comments)

# make predictions on the test data using the pre-trained logistic regression model
prediction = lg.predict(X_new)

# print the predicted labels for the test data
for i, (comment, pred_value) in enumerate(zip(comments, pred_sample_comments)):
    print("\nComment is '{}'.".format(comment))
    print("Predicted subreddit category is '{}'.".format(prediction[i]))
    print("Correct subreddit category is '{}'.".format(pred_value))



Comment is 'changed default stringsasfactors false r'.
Predicted subreddit category is 'datascience'.
Correct subreddit category is 'datascience'.

Comment is 'joke openai clown show rename closedai'.
Predicted subreddit category is 'MachineLearning'.
Correct subreddit category is 'MachineLearning'.

Comment is 'yeah simple script don’t think need sphnx'.
Predicted subreddit category is 'datascience'.
Correct subreddit category is 'datascience'.

Comment is 'phone screen good place find job really generally figure role entails phone interview varies widely job description states raise red flag lines salary job description analyst scientist roles pretty blurry right experience'.
Predicted subreddit category is 'datascience'.
Correct subreddit category is 'datascience'.

Comment is 'see specialized pytorchfocused env please ignore question'.
Predicted subreddit category is 'datascience'.
Correct subreddit category is 'MachineLearning'.
